<a href="https://colab.research.google.com/github/ollyekhan/SocialMediaMiningFinalProject/blob/nlp-topic-modeling-work/QA_Models_By_Ali_Bangash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing the data (TensorFlow)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [10]:
!pip install datasets evaluate transformers[sentencepiece]

In [11]:
!head dataset/postings.csv

"job_title","company","job_location","job_link","first_seen","search_city","search_country","job level","job_type","job_summary","job_skills"
"C# Software Engineer","E Tech Group","West Chester, OH","https://www.linkedin.com/jobs/view/c%23-software-engineer-at-e-tech-group-3780474203","2023-12-25","Covington","United States","Associate","Remote","At E Tech Group, joining our team means joining a group of passionate and forward-thinking experts. We’re one of the largest engineering and system integration firms in the United States providing value for our clients through IT automation and control solutions for more than 25 years to the Life Sciences, Mission Critical, Metals, Material Handling, Consumer Products and Food & Beverage industries. Our national scale allows us to help our clients develop and implement standardized solutions at any of their facilities regardless of physical location or the local resources available at that site.
We are seeking a
C# Software Engineer
to develop

In [14]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv("dataset/postings.csv")

# Dropping any NaN values in the about sections
df = df.dropna()
about_sections = df['job_skills'].tolist()

# Combine all about sections into one document
about_text = ' '.join(about_sections)

# Tokenize the text
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([about_text])

# Get the list of terms and their frequencies
terms = vectorizer.get_feature_names_out()
term_counts = X.toarray().sum(axis=0)

# Create a dictionary of terms and their counts
term_count_dict = dict(zip(terms, term_counts))

# Sort the terms by their counts in descending order
sorted_terms = sorted(term_count_dict.items(), key=lambda x: x[1], reverse=True)

# Display the top N most popular skills
top_skills_count = 1000  # Change this number to display more or fewer skills
top_skills = sorted_terms[:top_skills_count]

# Convert the list of tuples to a DataFrame for easier visualization
top_skills_df = pd.DataFrame(top_skills, columns=['Skill', 'Frequency'])

top_skills_df.to_csv('dataset/top_skills.csv', index=False)

print("Top Skills saved to top_skills.csv")

# now tokenize
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_data = tokenizer(about_sections, padding=True, truncation=True, return_tensors="tf")

# creating input sequences
input_sequences = {
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask']
}

# converting to tensorflow tensors
input_tensors = {key: tf.convert_to_tensor(value) for key, value in input_sequences.items()}

ParserError: Error tokenizing data. C error: EOF inside string starting at row 5403

In [13]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering

# Step 1: Read the CSV file
df = pd.read_csv('dataset/about_dataset.csv')

# Step 2: Preprocess the data (if needed)
# Example: Remove NaN values and select relevant columns
df = df.dropna()
text_data = df['about_section'].tolist()  # Assuming 'about_section' is the relevant column

# Step 3: Tokenization
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_data = tokenizer(text_data, padding=True, truncation=True, return_tensors="tf")

# Step 4: Create input sequences
# Example: Add special tokens and padding
input_sequences = {
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask']
}

# Step 5: Convert to TensorFlow tensors
input_tensors = {key: tf.convert_to_tensor(value) for key, value in input_sequences.items()}

# Load pre-trained question-answering model
qa_model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Define a question
question = "What are the most important skills?"

# Run the model to get answers
answers = qa_model(input_tensors)

# Process the answers
start_scores = tf.nn.softmax(answers.start_logits, axis=-1).numpy().tolist()
end_scores = tf.nn.softmax(answers.end_logits, axis=-1).numpy().tolist()

# Display the answers
for i, (start, end) in enumerate(zip(start_scores, end_scores)):
    all_tokens = tokenizer.convert_ids_to_tokens(tokenized_data['input_ids'][i].numpy())
    answer = ' '.join(all_tokens[start.index(max(start)):end.index(max(end))+1])
    print(f"About section {i+1}: {answer}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


ResourceExhaustedError: Exception encountered when calling layer 'self' (type TFBertSelfAttention).

{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[520,16,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BatchMatMulV2] name: 

Call arguments received by layer 'self' (type TFBertSelfAttention):
  • hidden_states=tf.Tensor(shape=(520, 512, 1024), dtype=float32)
  • attention_mask=tf.Tensor(shape=(520, 1, 1, 512), dtype=float32)
  • head_mask=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_value=None
  • output_attentions=False
  • training=False